In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np

## Help with this section of code

In [2]:
# Need to find a more pythonic way to load in this data
station_10 = pd.read_csv("~/Projects/NiceRide/Nice_Ride_data/2010/Nice_Ride_2010_station_locations.csv")
station_11 = pd.read_csv("~/Projects/NiceRide/Nice_Ride_data/2011/Nice_Ride_2011_station_locations.csv")
station_12 = pd.read_csv("~/Projects/NiceRide/Nice_Ride_data/2012/Nice_Ride_2012_station_locations.csv")
station_13 = pd.read_csv("~/Projects/NiceRide/Nice_Ride_data/2013/Nice_Ride_2013_station_locations.csv")
station_14 = pd.read_csv("~/Projects/NiceRide/Nice_Ride_data/2014/Nice_Ride_2014_station_locations.csv")
station_15 = pd.read_csv("~/Projects/NiceRide/Nice_Ride_data/2015/Nice_Ride_2015_station_locations.csv")
station_16 = pd.read_csv("~/Projects/NiceRide/Nice_Ride_data/2016/Nice_Ride_2016_station_locations.csv")
station_17 = pd.read_csv("~/Projects/NiceRide/Nice_Ride_data/2017/Nice_Ride_2017_station_locations.csv")

## Station 10

In [3]:
station_10.info()
station_10.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 5 columns):
Terminal     65 non-null int64
Station      65 non-null object
Latitude     65 non-null float64
Longitude    65 non-null float64
Nb docks     65 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 2.6+ KB


,Terminal,Station,Latitude,Longitude,Nb docks
0,30000,100 Main Street SE,44.984892,-93.256551,23
1,30001,25th Street & 33rd. Ave. S.,44.957340,-93.223740,15
2,30002,Augsburg College,44.966220,-93.238400,21
3,30003,Plymouth Ave N. & N. Oliver Ave,44.991412,-93.306569,23
4,30004,11th Street & Hennepin,44.975340,-93.278690,23


In [4]:
station_10 = station_10.rename(index=str, columns={"Nb docks": "Ndocks",})

## Station 11

In [5]:
station_11.info()
station_11.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 5 columns):
Terminal        116 non-null int64
Station         116 non-null object
Lat             116 non-null float64
Long            116 non-null float64
Install date    116 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 4.6+ KB


,Terminal,Station,Lat,Long,Install date
0,30000,100 Main Street SE,44.984892,-93.256551,4/8/2011
1,30001,25th Street & 33rd Ave. S,44.957340,-93.223740,4/8/2011
2,30002,Augsburg College,44.966220,-93.238400,4/8/2011
3,30003,Plymouth Ave. N & Oliver Ave. N,44.991412,-93.306269,4/9/2011
4,30004,11th Street & Hennepin,44.975340,-93.278690,4/8/2011


In [6]:
station_11 = station_11.rename(index=str, columns={'Lat': 'Latitude',
                                                  'Long': 'Longitude'})

In [7]:
del station_11['Install date']

2011 station data is the only year to explicitly include this data as a feature. Install date will be removed. The approximate install date can be derived from usage statistics if required.

## Station 12

In [8]:
station_12.info()
station_12.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 5 columns):
Terminal     146 non-null int64
Station      146 non-null object
Latitude     145 non-null float64
Longitude    145 non-null float64
Nb docks     145 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 5.8+ KB


,Terminal,Station,Latitude,Longitude,Nb docks
0,30000,100 Main Street SE,44.984892,-93.256551,27.0
1,30001,25th Street & 33rd. Ave. S.,44.957340,-93.223740,15.0
2,30002,Augsburg College,44.966220,-93.238400,15.0
3,30003,Plymouth Ave N. & N. Oliver Ave,44.991412,-93.306269,19.0
4,30004,11th Street & Hennepin,44.975340,-93.278690,23.0


There is a discrepency between the Terminal, Station, Latitude and Longitude, NDocks columns

In [9]:
station_12[station_12['Nb docks'].isna() == True]

,Terminal,Station,Latitude,Longitude,Nb docks
82,30082,Not installed in 2012,NaN,NaN,NaN


Terminal 30082 wasn't installed/used in 2012, and may be removed

In [10]:
station_12 = station_12.dropna()
station_12.reset_index(drop=True)
station_12.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145 entries, 0 to 145
Data columns (total 5 columns):
Terminal     145 non-null int64
Station      145 non-null object
Latitude     145 non-null float64
Longitude    145 non-null float64
Nb docks     145 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 6.8+ KB


In [11]:
station_12 = station_12.rename(index=str, columns={"Nb docks": "Ndocks"})
station_12['Ndocks'] = station_12['Ndocks'].astype(int)
station_12.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145 entries, 0 to 145
Data columns (total 5 columns):
Terminal     145 non-null int64
Station      145 non-null object
Latitude     145 non-null float64
Longitude    145 non-null float64
Ndocks       145 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 6.8+ KB


## Station 11 resolving Ndocks dilemna

Due to station 11 not having a Ndock column, we'll make one using the Ndock values of the previous year (2010), for any terminals that are new for 2011, we'll use the 2012 Ndock data to fill those values.

In [12]:
#?pd.merge()
temp = station_10.drop(['Station', 'Latitude', 'Longitude'], axis=1)
station_11 = pd.merge(station_11, temp, how='outer', on=('Terminal'))
temp = station_12.drop(['Station', 'Latitude', 'Longitude'], axis=1)
station_11 = pd.merge(station_11, temp, how='outer', on='Terminal')
station_11

,Terminal,Station,Latitude,Longitude,Ndocks_x,Ndocks_y
0,30000,100 Main Street SE,44.984892,-93.256551,23.0,27.0
1,30001,25th Street & 33rd Ave. S,44.957340,-93.223740,15.0,15.0
2,30002,Augsburg College,44.966220,-93.238400,21.0,15.0
3,30003,Plymouth Ave. N & Oliver Ave. N,44.991412,-93.306269,23.0,19.0
4,30004,11th Street & Hennepin,44.975340,-93.278690,23.0,23.0
5,30005,Hennepin & Central Ave. NE,44.988890,-93.253920,15.0,15.0
6,30006,YWCA Downtown,44.972217,-93.276435,19.0,23.0
7,30007,9th Street & 4th Ave. S,44.972650,-93.268720,15.0,19.0
8,30008,Midtown Exchange,44.948590,-93.260200,19.0,19.0
9,30009,4th Street & 13th Ave. SE,44.981020,-93.237460,15.0,15.0


In [13]:
# remove stations that existed in 2012 but not 2011
station_11 = station_11[station_11.Station.notna() == True]

In [14]:
station_11[station_11.Ndocks_y.isna() == True]

,Terminal,Station,Latitude,Longitude,Ndocks_x,Ndocks_y
82,30082,Wilder Street & Marshall Ave.,44.9486,-93.18583,NaN,NaN


One value in both Ndocks_x and Ndocks_y is NaN

In [15]:
print(np.mean(station_11.Ndocks_y))
print(np.mean(station_11.Ndocks_x))
print(np.mean(station_11[station_11.Ndocks_x.isna() == True].Ndocks_y))

17.321739130434782
18.415384615384614
15.32


The mean value for the station dock number is 17.32 and 18.41 for the 2010 and 2012 years. However, when looking at new stations (installed after 2010) the mean drops to 15.32. For this reason we'll go with 15.0 to fill the NaN value of Ndocks_y

In [16]:
station_11[station_11.Ndocks_y.isna() == True] # Find exact location of NaN
station_11.iloc[82,5] = 15.0 # Replace the value

# Use the 2012 values to fill the 2010 values that didn't exist
station_11 = station_11.fillna(value={'Ndocks_x': station_11[station_11.Ndocks_x.isna() == True].Ndocks_y})
station_11.info()

# station_11['Ndocks'] = station_11.apply(lambda row: np.mean(row['Ndocks_x'].value+row['Ndocks_y'].value), axis=1)
# station_11['Ndocks'] = np.mean(station_11.Ndocks_x+station_11.Ndocks_y) Computes the total mean
# station_11['Ndocks'] = np.mean(station_11.Ndocks_x.values+station_11.Ndocks_y.values) Does the same faster

# This next sections works like a hobgoblin, ugly but works
temp = station_11.drop(['Terminal', 'Station', 'Latitude', 'Longitude'], axis=1)
station_11['Ndocks'] = temp.agg('mean', axis=1)
del station_11['Ndocks_x']
del station_11['Ndocks_y']
station_11.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116 entries, 0 to 115
Data columns (total 6 columns):
Terminal     116 non-null int64
Station      116 non-null object
Latitude     116 non-null float64
Longitude    116 non-null float64
Ndocks_x     116 non-null float64
Ndocks_y     116 non-null float64
dtypes: float64(4), int64(1), object(1)
memory usage: 6.3+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 116 entries, 0 to 115
Data columns (total 5 columns):
Terminal     116 non-null int64
Station      116 non-null object
Latitude     116 non-null float64
Longitude    116 non-null float64
Ndocks       116 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 5.4+ KB


/home/grimoire/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
station_11.Ndocks = station_11.Ndocks.astype(int)

In [18]:
station_11

,Terminal,Station,Latitude,Longitude,Ndocks
0,30000,100 Main Street SE,44.984892,-93.256551,25
1,30001,25th Street & 33rd Ave. S,44.957340,-93.223740,15
2,30002,Augsburg College,44.966220,-93.238400,18
3,30003,Plymouth Ave. N & Oliver Ave. N,44.991412,-93.306269,21
4,30004,11th Street & Hennepin,44.975340,-93.278690,23
5,30005,Hennepin & Central Ave. NE,44.988890,-93.253920,15
6,30006,YWCA Downtown,44.972217,-93.276435,21
7,30007,9th Street & 4th Ave. S,44.972650,-93.268720,17
8,30008,Midtown Exchange,44.948590,-93.260200,19
9,30009,4th Street & 13th Ave. SE,44.981020,-93.237460,15


## Station 13

In [19]:
station_13.info()
station_13.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 6 columns):
Terminal      170 non-null int64
Station       170 non-null object
Latitude      170 non-null float64
Longitude     170 non-null float64
Nb docks      170 non-null int64
Unnamed: 5    1 non-null object
dtypes: float64(2), int64(2), object(2)
memory usage: 8.0+ KB


,Terminal,Station,Latitude,Longitude,Nb docks,Unnamed: 5
0,30000,100 Main Street SE,44.984892,-93.256551,27,NaN
1,30001,25th Street & 33rd. Ave. S.,44.957340,-93.223740,15,NaN
2,30002,Augsburg College,44.966220,-93.238400,15,NaN
3,30003,Plymouth Ave N. & N. Oliver Ave,44.991412,-93.306269,15,NaN
4,30004,11th Street & Hennepin,44.975340,-93.278690,23,NaN


**Before we remove the Unnamed: 5 column lets figure out what non-NaN values it has.**

In [20]:
station_13[station_13['Unnamed: 5'].notna()]

,Terminal,Station,Latitude,Longitude,Nb docks,Unnamed: 5
167,30167,LaSalle Ave & 9th Street S (10th & Nicollet),44.975504,-93.275513,19,"Station was at 10th & Nicollet (44.97384, -93...."


It contains a note that the station was at a different location, this information is already stored in the station name, and in the lat/long difference from previous year. This column is redundant and will be removed.

In [21]:
del station_13['Unnamed: 5']

In [22]:
station_13 = station_13.rename(index=str, columns={"Nb docks": "Ndocks"})

## Station 14

In [23]:
station_14.info()
station_14.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 5 columns):
Terminal     170 non-null int64
Station      170 non-null object
Latitude     169 non-null float64
Longitude    169 non-null float64
Nb Docks     170 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 6.7+ KB


,Terminal,Station,Latitude,Longitude,Nb Docks
0,30000,100 Main Street SE,44.984892,-93.256551,27
1,30001,25th Street & 33rd Ave S,44.957341,-93.223374,15
2,30002,Riverside Ave & 23rd Ave S,44.967115,-93.240149,15
3,30003,Plymouth Ave N & N Oliver Ave,44.991412,-93.306269,15
4,30004,11th Street & Hennepin,44.975340,-93.278690,23


There is a discrepancy between Terminal, Station, Nb Docks and Latitude, Longitude

In [24]:
station_14[station_14.Longitude.isna() == True]

,Terminal,Station,Latitude,Longitude,Nb Docks
167,30167,Not installed in 2014,NaN,NaN,0


Terminal 30167 wasn't install in 2014, this observation will be removed

In [25]:
station_14 = station_14.dropna()
station_14.reset_index(drop=True)
station_14.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169 entries, 0 to 169
Data columns (total 5 columns):
Terminal     169 non-null int64
Station      169 non-null object
Latitude     169 non-null float64
Longitude    169 non-null float64
Nb Docks     169 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 7.9+ KB


In [26]:
station_14 = station_14.rename(index=str, columns={"Nb Docks": "Ndocks"})

## Station 15

In [27]:
station_15.info()
station_15.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 6 columns):
Terminal     190 non-null int64
Station      190 non-null object
Latitude     190 non-null float64
Longitude    190 non-null float64
NbDocks      190 non-null int64
Notes        22 non-null object
dtypes: float64(2), int64(2), object(2)
memory usage: 9.0+ KB


,Terminal,Station,Latitude,Longitude,NbDocks,Notes
0,30000,100 Main Street SE,44.984892,-93.256551,27,NaN
1,30001,25th Street & 33rd Ave S,44.957341,-93.223374,15,NaN
2,30002,Riverside Ave & 23rd Ave S,44.967115,-93.240149,15,NaN
3,30003,Plymouth Ave N & N Oliver Ave,44.991412,-93.306269,15,NaN
4,30004,11th Street & Hennepin,44.975340,-93.278690,23,NaN


In [28]:
station_15 = station_15.rename(index=str, columns={"NbDocks": "Ndocks",})

In [29]:
station_15[station_15['Notes'].notna()]

,Terminal,Station,Latitude,Longitude,Ndocks,Notes
55,30055,Malcolm Ave SE & 5th Street SE,44.973371,-93.211264,25,"Moved 6/4/15 from 4th and Hennepin 44.980159,..."
169,30169,Marshall Street NE & 18th Ave NE,45.006229,-93.271571,15,Approx. Install Dates
170,30170,Nicollet Island,44.985388,-93.261450,15,7/22/2015
171,30171,SE 3rd Ave & University Ave SE,44.985465,-93.252102,15,7/23/2015
172,30172,2nd Street SE & 5th Ave SE,44.983321,-93.249653,15,7/23/2015
173,30173,22nd Ave NE & California Street NE,45.010119,-93.268598,15,8/6/2015
174,30174,Fort Snelling LRT Station,44.892483,-93.197820,15,8/27/2015
175,30175,18th Ave Trail & 2nd Street NE,45.006321,-93.266553,15,8/7/2015
176,30176,Washington Ave N & Cedar Lake Trail,44.985406,-93.274031,19,8/19/2015
177,30177,3rd Street NE & Lowry Ave NE,45.013040,-93.264573,15,8/22/2105


Notes include install dates, usage data can show approximately when the station became in use. There was a station that was moved this year. Due to the fact that we don't have the data on when it was moved, usage data may be able to determine this information. For the purposes that we're using the data for, this isn't necessary information and can be dropped.

In [30]:
del station_15['Notes']
station_15.info()

<class 'pandas.core.frame.DataFrame'>
Index: 190 entries, 0 to 189
Data columns (total 5 columns):
Terminal     190 non-null int64
Station      190 non-null object
Latitude     190 non-null float64
Longitude    190 non-null float64
Ndocks       190 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 8.9+ KB


## Station 16

In [31]:
station_16.info()
station_16.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 6 columns):
Terminal      202 non-null object
Station       202 non-null object
Latitude      202 non-null float64
Longitude     202 non-null float64
Nb Docks      202 non-null int64
Unnamed: 5    4 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 9.5+ KB


,Terminal,Station,Latitude,Longitude,Nb Docks,Unnamed: 5
0,30000,100 Main Street SE,44.984892,-93.256551,27,NaN
1,30039,10th Street & Nicollet Mall,44.973839,-93.274544,11,NaN
2,30117,10th Street E & Cedar Street,44.950189,-93.097590,15,NaN
3,30154,11th Ave S & S 2nd Street,44.976730,-93.252189,23,NaN
4,30004,11th Street & Hennepin,44.975340,-93.278690,23,NaN


In [32]:
station_16[station_16['Unnamed: 5'].notna()]

,Terminal,Station,Latitude,Longitude,Nb Docks,Unnamed: 5
96,30084-A,Hennepin Ave & N 5th Street,44.979550,-93.272883,15,Moved from Grand & Cretin on Aug 19th
97,30084,Grand & Cretin,44.939936,-93.192758,15,Moved Aug 19th to Hennepin Ave & N 5th Street
149,30146-A,North Mississippi Regional Park,45.042435,-93.283734,19,Moved form Webber Park on Aug 9th
150,30146,Webber park,45.033069,-93.290650,15,Moved Aug 9th North Mississippi Regional Park


The moved data here does include note on the date it was moved. For simplicity of analysis, we'll remove the original location "xxxxx-A". This will allow us to have the terminal ID's as ints, and remove the Unnamed: 5 column.

In [33]:
del station_16['Unnamed: 5']

In [34]:
station_16.drop(station_16.index[149], inplace = True)
station_16.drop(station_16.index[96], inplace = True)

In [35]:
np.sort(station_16.Terminal.values)

array(['30000', '30001', '30002', '30003', '30004', '30005', '30006',
       '30007', '30008', '30009', '30010', '30011', '30012', '30013',
       '30014', '30015', '30016', '30017', '30018', '30019', '30020',
       '30021', '30022', '30023', '30024', '30025', '30026', '30027',
       '30028', '30029', '30030', '30031', '30032', '30033', '30034',
       '30035', '30036', '30037', '30038', '30039', '30040', '30041',
       '30042', '30043', '30044', '30045', '30046', '30047', '30048',
       '30049', '30050', '30051', '30052', '30053', '30054', '30055',
       '30056', '30057', '30058', '30059', '30060', '30061', '30062',
       '30063', '30064', '30065', '30066', '30067', '30068', '30069',
       '30070', '30071', '30072', '30073', '30074', '30075', '30076',
       '30077', '30078', '30079', '30080', '30081', '30082', '30083',
       '30084', '30085', '30086', '30087', '30088', '30089', '30090',
       '30091', '30092', '30093', '30094', '30095', '30096', '30097',
       '30098', '300

Need to determine what to do with 'NRHQ' terminal ID

"We've had the station at our Headquarters on and off over the years, it's just been the last couple years that we've placed it on our maps. The reason it doesn't have a standing number is because we pretty much use whatever extra station parts we have after installing all the other stations to place some docks out in front of our offices. The station here, is not always the same station." - NiceRideMN

This isn't a commercial terminal, and will be removed from this and subsequent data

In [36]:
station_16 = station_16[station_16.Terminal != 'NRHQ ']

In [37]:
station_16.Terminal = station_16.Terminal.astype(int)

In [38]:
station_16 = station_16.rename(index=str, columns={"Nb Docks": "Ndocks"})

In [39]:
station_16.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199 entries, 0 to 201
Data columns (total 5 columns):
Terminal     199 non-null int64
Station      199 non-null object
Latitude     199 non-null float64
Longitude    199 non-null float64
Ndocks       199 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 9.3+ KB


## Station 17

In [40]:
station_17.info()
station_17.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 5 columns):
Number         202 non-null object
Name           202 non-null object
Latitude       202 non-null float64
Longitude      202 non-null float64
Total docks    202 non-null int64
dtypes: float64(2), int64(1), object(2)
memory usage: 8.0+ KB


,Number,Name,Latitude,Longitude,Total docks
0,30000,100 Main Street SE,44.984892,-93.256551,27
1,30001,25th Street & 33rd Ave S,44.957341,-93.223374,15
2,30002,Riverside Ave & 23rd Ave S,44.967115,-93.240149,15
3,30003,Plymouth Ave N & N Oliver Ave,44.991412,-93.306269,15
4,30004,11th Street & Hennepin,44.975340,-93.278690,23


In [41]:
station_17 = station_17.rename(index=str, columns={'Total docks': 'Ndocks',
                                                  'Name': 'Station',
                                                  'Number': 'Terminal'})

In [42]:
station_17.info()

<class 'pandas.core.frame.DataFrame'>
Index: 202 entries, 0 to 201
Data columns (total 5 columns):
Terminal     202 non-null object
Station      202 non-null object
Latitude     202 non-null float64
Longitude    202 non-null float64
Ndocks       202 non-null int64
dtypes: float64(2), int64(1), object(2)
memory usage: 9.5+ KB


Terminal column is of type object, need to investigate further:

In [43]:
station_17.Terminal.values

array(['30000', '30001', '30002', '30003', '30004', '30005', '30006',
       '30007', '30008', '30009', '30010', '30011', '30012', '30013',
       '30014', '30015', '30016', '30017', '30018', '30019', '30020',
       '30021', '30022', '30023', '30024', '30025', '30026', '30027',
       '30028', '30029', '30030', '30031', '30032', '30033', '30034',
       '30035', '30037', '30038', '30039', '30040', '30041', '30042',
       '30043', '30044', '30045', '30046', '30047', '30048', '30049',
       '30050', '30051', '30052', '30053', '30054', '30055', '30056',
       '30057', '30058', '30059', '30060', '30061', '30062', '30063',
       '30064', '30065', '30066', '30067', '30068', '30069', '30070',
       '30071', '30073', '30074', '30075', '30076', '30077', '30078',
       '30079', '30080', '30081', '30082', '30083', '30085', '30086',
       '30087', '30088', '30089', '30090', '30091', '30092', '30093',
       '30094', '30095', '30096', '30097', '30098', '30099', '30100',
       '30101', '301

In [44]:
# Removing NRHQ row and changing from dtype object to int
station_17 = station_17[station_17.Terminal != 'NRHQ']
station_17.Terminal = station_17.Terminal.astype(int)

In [45]:
station_17.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201 entries, 0 to 200
Data columns (total 5 columns):
Terminal     201 non-null int64
Station      201 non-null object
Latitude     201 non-null float64
Longitude    201 non-null float64
Ndocks       201 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 9.4+ KB


In [46]:
print(station_10.info())
print(station_11.info())
print(station_12.info())
print(station_13.info())
print(station_14.info())
print(station_15.info())
print(station_16.info())
print(station_17.info())

<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, 0 to 64
Data columns (total 5 columns):
Terminal     65 non-null int64
Station      65 non-null object
Latitude     65 non-null float64
Longitude    65 non-null float64
Ndocks       65 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 3.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 116 entries, 0 to 115
Data columns (total 5 columns):
Terminal     116 non-null int64
Station      116 non-null object
Latitude     116 non-null float64
Longitude    116 non-null float64
Ndocks       116 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 5.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 145 entries, 0 to 145
Data columns (total 5 columns):
Terminal     145 non-null int64
Station      145 non-null object
Latitude     145 non-null float64
Longitude    145 non-null float64
Ndocks       145 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 6.8+ KB
None
<class 'p

In [47]:
station_10.to_csv("~/Projects/NiceRide/Nice_Ride_data/2010/NiceRide_station_2010.csv", index=False)
station_11.to_csv("~/Projects/NiceRide/Nice_Ride_data/2011/NiceRide_station_2011.csv", index=False)
station_12.to_csv("~/Projects/NiceRide/Nice_Ride_data/2012/NiceRide_station_2012.csv", index=False)
station_13.to_csv("~/Projects/NiceRide/Nice_Ride_data/2013/NiceRide_station_2013.csv", index=False)
station_14.to_csv("~/Projects/NiceRide/Nice_Ride_data/2014/NiceRide_station_2014.csv", index=False)
station_15.to_csv("~/Projects/NiceRide/Nice_Ride_data/2015/NiceRide_station_2015.csv", index=False)
station_16.to_csv("~/Projects/NiceRide/Nice_Ride_data/2016/NiceRide_station_2016.csv", index=False)
station_17.to_csv("~/Projects/NiceRide/Nice_Ride_data/2017/NiceRide_station_2017.csv", index=False)

In [48]:
# Make an empty DefaultDict
# fill dictionary {year: station data}
# build hierarchal dataframe using dictionary


nr_dict = defaultdict()
for x in [2010 + x for x in range(8)] :
    nr_dict[x] = pd.read_csv("~/Projects/NiceRide/Nice_Ride_data/"+str(x)+"/NiceRide_station_"+str(x)+".csv")
NR = pd.concat(nr_dict)

In [49]:
nr_dict

defaultdict(None,
            {2010:     Terminal                           Station   Latitude  Longitude  Ndocks
             0      30000                100 Main Street SE  44.984892 -93.256551      23
             1      30001       25th Street & 33rd. Ave. S.  44.957340 -93.223740      15
             2      30002                  Augsburg College  44.966220 -93.238400      21
             3      30003  Plymouth Ave N. &  N. Oliver Ave  44.991412 -93.306569      23
             4      30004            11th Street & Hennepin  44.975340 -93.278690      23
             5      30005          Hennepin & 5th Street NE  44.988890 -93.253920      15
             6      30006                     YWCA Downtown  44.972217 -93.276350      19
             7      30007             9th Street & 4th Ave.  44.972650 -93.268720      15
             8      30008                  Midtown Exchange  44.949130 -93.261240      19
             9      30009         4th Street & 13th Ave. SE  44.981020 -93.2

In [50]:
NR

Terminal                                Station   Latitude  \
2010 0       30000                     100 Main Street SE  44.984892   
     1       30001            25th Street & 33rd. Ave. S.  44.957340   
     2       30002                       Augsburg College  44.966220   
     3       30003       Plymouth Ave N. &  N. Oliver Ave  44.991412   
     4       30004                 11th Street & Hennepin  44.975340   
     5       30005               Hennepin & 5th Street NE  44.988890   
     6       30006                          YWCA Downtown  44.972217   
     7       30007                  9th Street & 4th Ave.  44.972650   
     8       30008                       Midtown Exchange  44.949130   
     9       30009              4th Street & 13th Ave. SE  44.981020   
     10      30010                 Hennepin & Laurel Ave.  44.973910   
     11      30011          North 2nd Street & 3rd Ave. N  44.986078   
     12      30012                Mpls. Convention Center  44.970098   
     13      30013                  Lake Street & Lyndale  44.949640   
     14      30014                  Franklin & 21st. Ave.  44.962670   
     15      30015                     Washington & Cedar  44.973440   
     16      30016                      Walker Art Center  44.967460   
     17      30017                  22nd Street & Lyndale  44.960744   
     18      30018                                   MCAD  44.957280   
     19      30019                            Blegen Hall  44.972169   
     20      30020                            YWCA Uptown  44.951310   
     21      30021                Washington & 2nd Ave. N  44.983830   
     22      30022                  26th Street & Lyndale  44.955480   
     23      30023                  Chicago & 27th Street  44.953396   
     24      30024                          Kolthoff Hall  44.973799   
     25      30025                        Williamson Hall  44.977406   
     26      30026                              Lind Hall  44.974190   
     27      30027                         Wilson Library  44.970653   
     28      30028                 Washington & Marquette  44.981120   
     29      30029                5th Street & 3rd Ave. S  44.977390   
...            ...                                    ...        ...   
2017 171     30174           Hiawatha Ave & E 50th Street  44.912943   
     172     30175         18th Ave Trail & 2nd Street NE  45.006321   
     173     30176    Washington Ave N & Cedar Lake Trail  44.985406   
     174     30177           3rd Street NE & Lowry Ave NE  45.013040   
     175     30178             5th Street NE & 1st Ave NE  44.990328   
     176     30179                Park Ave & 4th Street S  44.975895   
     177     30180               3rd Street N & 8th Ave N  44.987996   
     178     30181               5th Ave N & 4th Street N  44.984460   
     179     30182                           Sanford Hall  44.980831   
     180     30183                     Weisman Art Museum  44.973607   
     181     30184        Hennepin Ave & S Washington Ave  44.981895   
     182     30185             5th Street NE & 3rd Ave NE  44.991821   
     183     30186           S 5th Street & Nicollet Mall  44.978665   
     184     30187          Groveland Ave & Clifton Place  44.966145   
     185     30188               S 8th Street & 2nd Ave S  44.974795   
     186     30189       Delaware Street SE & 26th Ave SE  44.972128   
     187     30190            7th Street SE & 10th Ave SE  44.985576   
     188     30191  University Ave SE & Bedford Street SE  44.968760   
     189     30192                             Cooke Hall  44.976303   
     190     30193                         Mariucci Arena  44.977899   
     191     30194        23rd Ave SE & University Ave SE  44.974397   
     192     30195                            Willey Hall  44.972404   
     193     30196                      Civil Engineering  44.976096   
     194     30197            15th Ave SE & 4th St